<a href="https://colab.research.google.com/github/remi-boivin/own-gpt/blob/google_colab/own-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 1000
eval_iters = 50
dropout=0.2
n_embd = 384

cuda


wocabulary

In [36]:
with open("wizard_of_oz.txt", encoding="utf-8") as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', '\ufeff']


encoder

In [37]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([83,  2, 72, 57, 81, 68, 71, 57, 60,  2, 23, 83,  2, 57, 68, 68, 45, 64,
        71, 74, 76, 59, 77, 76, 75, 31, 70, 57, 58, 68, 61, 60,  2, 23, 62, 57,
        68, 75, 61,  9,  2, 62, 65, 68, 61, 46, 74, 61, 61,  2, 23, 83,  2,  2,
        23, 83,  2, 65, 76, 61, 69, 75,  2, 23, 53, 83,  2, 70, 57, 69, 61,  2,
        23,  2, 44, 31, 27, 30, 39, 31, 11, 69, 60,  2,  9,  2, 72, 57, 76, 64,
         2, 23,  2, 44, 31, 27, 30, 39, 31, 11])


In [38]:
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else valid_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
#  print(ix)
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

x,y = get_batch('train')
print(f"inputs: {x}\n targets {y}")

RuntimeError: ignored

In [23]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
      out[split] = losses.mean()
  model.train()
  return out

In [34]:
class GPTLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, index, max_new_token):
    # index is (B, T) array of indices in the current context
    for _ in range(max_new_token):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the latest time step
      logits = logits[:, -1, :] # become (B, C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim= 1) # (B, T + 1)
    return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_token=3500)[0].tolist())
print(generated_chars)


RuntimeError: ignored

In [25]:
#x = train_data[:block_size]
#y = train_data[1:block_size + 1]

#for t in range(block_size):
#  context = x[:t + 1]
#  target = y[t]
#  print('when input is', context, 'target is', target)
import time
started_time = time.time()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
  xb, yb = get_batch('train')

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
end_time = time.time()
print(f"time of training: {end_time - started_time}\nloss: {loss.item()}")


step: 0, train loss: 5.021, val loss: 4.738
step: 50, train loss: 5.015, val loss: 4.774
step: 100, train loss: 5.009, val loss: 4.696
step: 150, train loss: 4.967, val loss: 4.757
step: 200, train loss: 4.982, val loss: 4.704
step: 250, train loss: 4.906, val loss: 4.687
step: 300, train loss: 4.941, val loss: 4.634
step: 350, train loss: 4.921, val loss: 4.710
step: 400, train loss: 4.930, val loss: 4.734
step: 450, train loss: 4.859, val loss: 4.650
step: 500, train loss: 4.884, val loss: 4.704
step: 550, train loss: 4.886, val loss: 4.718
step: 600, train loss: 4.880, val loss: 4.603
step: 650, train loss: 4.838, val loss: 4.639
step: 700, train loss: 4.789, val loss: 4.595
step: 750, train loss: 4.806, val loss: 4.618
step: 800, train loss: 4.812, val loss: 4.648
step: 850, train loss: 4.780, val loss: 4.596
step: 900, train loss: 4.759, val loss: 4.592
step: 950, train loss: 4.750, val loss: 4.562
step: 1000, train loss: 4.750, val loss: 4.581
step: 1050, train loss: 4.710, val l

In [15]:
x = torch.tensor([-0.05], dtype=torch.float32)
y = F.sigmoid(x)
print(y)

tensor([0.4875])
